In [ ]:
from ipyleaflet import Map, GeomanDrawControl, basemaps

south_lat = float(input("Enter south latitude: "))
west_lon = float(input("Enter west longitude: "))
north_lat = float(input("Enter north latitude: "))
east_lon = float(input("Enter east longitude: "))

x = (south_lat + north_lat)/2
y = (east_lon + west_lon)/2

m = Map(basemap=basemaps.Stadia.StamenTerrain, center=(x,y), zoom=5)

draw_control = GeomanDrawControl()
draw_control.polyline =  {
    "pathOptions": {
        "color": "#6bc2e5",
        "weight": 8,
        "opacity": 1.0
    }
}
draw_control.polygon = {
    "pathOptions": {
        "fillColor": "#6be5c3",
        "color": "#6be5c3",
        "fillOpacity": 1.0
    }
}
draw_control.circlemarker = {
    "pathOptions": {
        "fillColor": "#efed69",
        "color": "#efed69",
        "fillOpacity": 0.62
    }
}
draw_control.rectangle = {
    "pathOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 1.0
    }
} 

feature_collection = {
    'type': 'FeatureCollection',
    'features': []
}

In [ ]:
from shapely.geometry import shape

def handle_draw(self, action, geo_json):
    feature_collection['features'].append(geo_json)

draw_control.on_draw(handle_draw)
    
print("Draw a shape on the map now.")

In [ ]:
m.add_control(draw_control)
m

In [ ]:
if feature_collection['features']:
    coordinates = feature_collection['features'][-1][0]['geometry']['coordinates'][0]
    print(coordinates)
else:
    print("No shapes drawn yet.")

In [ ]:
from shapely.geometry import Polygon
polygon = Polygon(coordinates)
print(polygon)

In [ ]:
import openeo
from shapely.geometry import Polygon

polygon = Polygon(coordinates)

connection = openeo.connect("openeofed.dataspace.copernicus.eu")
connection.authenticate_oidc()

datacube = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent=polygon,
    temporal_extent=["2025-04-28", "2025-04-30"],
    bands=["B04", "B08", "SCL"],
    max_cloud_cover=20,
)

red = datacube.band("B04") * 0.0001
nir = datacube.band("B08") * 0.0001
scl = datacube.band("SCL")

ndvi = (nir - red) / (nir + red)

valid_data_mask = (scl == 2) | (scl == 4)
valid_data_mask = valid_data_mask.resample_cube_spatial(datacube)

masked_ndvi = ndvi.mask(valid_data_mask)
max_ndvi = masked_ndvi.max_time()

job = max_ndvi.execute_batch()
job.start_and_wait()
job.download_results("Border_Image.geotiff")

In [ ]:
import rasterio
import matplotlib.pyplot as plt
import os

with rasterio.open("C:/Users/todor/Data_visualisation/Border_Image.geotiff") as src:
    ndvi = src.read(1)
    print(src.shape)

plt.imsave('Image.png', ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
print("Saved as Exercise_Image.png")

In [ ]:
from ipyleaflet import ImageOverlay

image = ImageOverlay(
    url="C:/Users/todor/Data_visualisation/Exercise_Image.png",
    bounds=coordinates
)

if rectangle:
    m.add(image)
m